In [ ]:
len(df), len(df3)

In [ ]:
df3.head()

In [ ]:
import colorlover as cl
bupu = cl.scales['9']['seq']['BuPu']
bupu = cl.interp( bupu, 48 )[32:48]
len(bupu)

In [1]:
df3['price'].unique()

NameError: name 'df3' is not defined

In [ ]:
levels = ['228.33', '292.31', '262.08', '182.25', \
         '189.58', '340.9', '198.26', '138.03']

color_match = dict(zip(levels, bupu))
print(color_match)

In [6]:
plot_data3 = []
for index,row in df3.iterrows():
    if df3['geometry'][index].type == 'Polygon':
        x,y = row.geometry.exterior.xy
        c_x,c_y = row.geometry.centroid.xy
    elif df3['geometry'][index].type == 'MultiPolygon':
        x = [poly.exterior.xy[0] for poly in df3['geometry'][index]]
        y = [poly.exterior.xy[1] for poly in df3['geometry'][index]]
        x_c = [poly.centroid.xy[0] for poly in df3['geometry'][index]]
        y_c = [poly.centroid.xy[1] for poly in df3['geometry'][index]]        
    else: 
        print('stop')
    ward_outline = dict(
            type = 'scatter',
            showlegend = False,
            legendgroup = "shapes",
            line = dict(color='black', width=1),
            x=x,
            y=y,
            fill='toself',
            fillcolor = color_match[str(row['price'])],
            hoverinfo='none'
    ) 
    hover_point = dict(
            type = 'scatter',
            showlegend = False,
            legendgroup = "centroids",
            name = row.NAME,
            text = 'Average Price: ' + str(row['price']),
            marker = dict(size=2, color='white'),
            x=c_x,
            y=c_y,
            fill='toself',
    )
    plot_data3.append(ward_outline)
    plot_data.append(hover_point)

NameError: name 'df3' is not defined

In [ ]:
fig = dict(data=plot_data3, layout=layout)
py.plot(fig, filename='DC_ward_Airbnb.html')

In [ ]:
total_points = 0
for trace in plot_data3:
    total_points = total_points + len(trace['x'])
print(total_points)

In [ ]:
SIMPLIFY_FACTOR = 0.05

plot_data_simplified = []
x_centroids = []
y_centroids = []
centroid_text = []
for index, row in df3.iterrows():
    level = str(row['price'])
    if df3['geometry'][index].type == 'Polygon':
        x = row.geometry.simplify(SIMPLIFY_FACTOR).exterior.xy[0].tolist()
        y = row.geometry.simplify(SIMPLIFY_FACTOR).exterior.xy[1].tolist()
        x_c, y_c = row.geometry.centroid.xy
        t_c = row.NAME + '<br>Price: ' + level      
        x_centroids.append(x_c[0])
        y_centroids.append(y_c[0])
        centroid_text.append(t_c)        
    elif df3['geometry'][index].type == 'MultiPolygon':
        x = [poly.simplify(SIMPLIFY_FACTOR).exterior.xy[0] for poly in df3['geometry'][index]]
        y = [poly.simplify(SIMPLIFY_FACTOR).exterior.xy[1] for poly in df3['geometry'][index]]
        x_c = [poly.centroid.xy[0] for poly in df3['geometry'][index]]
        y_c = [poly.centroid.xy[1] for poly in df3['geometry'][index]]
        text = row.NAME + '<br>Price: ' + level
        t_c = [text for poly in df3['geometry'][index]]
        x_centroids = x_c + x_centroids
        y_centroids = y_c + y_centroids
        centroid_text = t_c + centroid_text
    else: 
        print('stop')
    x_traces[level] = x_traces[level] + x + [np.nan]
    y_traces[level] = y_traces[level] + y + [np.nan]


for lev in levels:
    county_outline = dict(
            type = 'scatter',
            line = dict(color='black', width=0.5),
            name = lev,
            x = x_traces[lev],
            y = y_traces[lev],
            fill = 'toself',
            fillcolor = cubic_helix[lev],
            hoverinfo='none',
            mode='lines'
    )
    plot_data_simplified.append(county_outline)
    
hover_points = dict(
        type = 'scatter',
        showlegend = False,
        legendgroup = "centroids",
        x = x_centroids,
        y = y_centroids,
        text = 'Average Price: ' + str(row['price']),       
        name = 'DC Wards',
        marker = dict(size=2, color='white'),
        mode='markers',
)

plot_data_simplified.append(hover_points)